In [1]:
#AES Encryption for a single block
#input: plaintext & key 128 bits / 16 bytes/ 32 hex digits

import nbimporter,os
from AES_key_schedule import key_schedule
class aes_encrypt():
    
    #initialization
    def __init__(self,plain_32hex,key_32hex):
        #row and column are transposed
        self.statearray = [[None for x in range(4)] for x in range(4)]
        self.plain_32hex = plain_32hex
        self.key_32hex = key_32hex
        
        self.mix_matrix = [
            ['02','03','01','01'],
            ['01','02','03','01'],
            ['01','01','02','03'],
            ['03','01','01','02']
        ]
        self.sbox = [
            ['63','7c','77','7b','f2','6b','6f','c5','30','01','67','2b','fe','d7','ab','76'],
            ['ca','82','c9','7d','fa','59','47','f0','ad','d4','a2','af','9c','a4','72','c0'], 
            ['b7','fd','93','26','36','3f','f7','cc','34','a5','e5','f1','71','d8','31','15'],
            ['04','c7','23','c3','18','96','05','9a','07','12','80','e2','eb','27','b2','75'],
            ['09','83','2c','1a','1b','6e','5a','a0','52','3b','d6','b3','29','e3','2f','84'],
            ['53','d1','00','ed','20','fc','b1','5b','6a','cb','be','39','4a','4c','58','cf'], 
            ['d0','ef','aa','fb','43','4d','33','85','45','f9','02','7f','50','3c','9f','a8'],
            ['51','a3','40','8f','92','9d','38','f5','bc','b6','da','21','10','ff','f3','d2'], 
            ['cd','0c','13','ec','5f','97','44','17','c4','a7','7e','3d','64','5d','19','73'],
            ['60','81','4f','dc','22','2a','90','88','46','ee','b8','14','de','5e','0b','db'],
            ['e0','32','3a','0a','49','06','24','5c','c2','d3','ac','62','91','95','e4','79'],
            ['e7','c8','37','6d','8d','d5','4e','a9','6c','56','f4','ea','65','7a','ae','08'],
            ['ba','78','25','2e','1c','a6','b4','c6','e8','dd','74','1f','4b','bd','8b','8a'],
            ['70','3e','b5','66','48','03','f6','0e','61','35','57','b9','86','c1','1d','9e'],
            ['e1','f8','98','11','69','d9','8e','94','9b','1e','87','e9','ce','55','28','df'], 
            ['8c','a1','89','0d','bf','e6','42','68','41','99','2d','0f','b0','54','bb','16']

        ]
    def init_statearray(self):
        quotient = int(len(self.plain_32hex)/2)
        m,n = 0,0
        for i in range(quotient):
            item = self.plain_32hex[2*i] + self.plain_32hex[2*i+1]
            
            if None in self.statearray[n]:
                self.statearray[n][m] = item 
                m = m + 1
            else:
                self.statearray[n+1][0] = item
                n,m = n+1,1
            
    def key_expansion(self):
        a = key_schedule(self.key_32hex)
        keys = a.key_expansion()
        return keys
    
    def transpose(self,matrix):
        t = [[None]*len(matrix) for row in range(len(matrix[0]))]
        for i in range(len(matrix[0])):
            for j in range(len(matrix)):
                t[i][j] = matrix[j][i]
        return t

    def bin_to_hex(self,bit):
        bin_to_decimal = int(bit,2)
        decimal_to_hex = hex(bin_to_decimal)[2:].zfill(2)
        return decimal_to_hex.upper()

    def hex_to_bin(self,_hex):
        hex_to_decimal = int(_hex,16)
        decimal_to_bin = bin(hex_to_decimal)[2:].zfill(8)
        return decimal_to_bin

    def XOR(self,hex1,hex2):
        bit1 = self.hex_to_bin(hex1)
        bit2 = self.hex_to_bin(hex2)
        xor_result = ''
        for index in range(len(bit1)):
            r = (int(bit1[index])+int(bit2[index])) % 2
            xor_result = xor_result + str(r) 
        xor_result_hex = self.bin_to_hex(xor_result) 
        return xor_result_hex    
    
    def GF_multiply(self,hex1,hex2):
        bit1 = bin(int(hex1,16))[2:]
        bit2 = bin(int(hex2,16))[2:]
        coeffi_poly1,coeffi_poly2 = list(),list()
        
        #reverse the order of coefficient
        for i in range(1,len(list(bit1))+1):
            coeffi_poly1.append(int(list(bit1)[-i]))
        for j in range(1,len(list(bit2))+1):
            coeffi_poly2.append(int(list(bit2)[-j]))
        
        #multiply two polynomials 
        len1,len2 = len(coeffi_poly1),len(coeffi_poly2)
        muilt_two_poly= [0 for i in range(len1+len2-1)]
        for i in range(len1):
            for j in range(len2):
                muilt_two_poly[i+j] += coeffi_poly1[i]*coeffi_poly2[j] 
        #after_mod2
        current = [i%2 for i in muilt_two_poly]
        
        #mod irreducible poly: x^8+x^4+x^3+x+1
        irr_poly = [1,1,0,1,1,0,0,0,1]
        
        while len(current) >= len(irr_poly):
            mod_poly = [i for i in irr_poly]
            for i in range(len(current) - len(irr_poly)):
                mod_poly.insert(0,0)
            #xor two ploynomials
            for j in range(len(current)):
                current[j] = (current[j] + mod_poly[j]) % 2
            #drop the 0 coefficient (let the last item be the highest term)
            while not current[-1]:
                current.pop()              

        #reverse the order of coefficient
        temp = [i for i in current]
        for i in range(len(temp)):
            current[i] = temp[-(i+1)]
            
        #transform into hex digits
        bit_string = ''.join([str(i) for i in current]).zfill(8)
        result_hex = self.bin_to_hex(bit_string)
        return result_hex 
        
    '''encryption'''
    def generate(self):
        self.init_statearray()
        key_set = self.key_expansion()
        
        #round_0: adding_roundkey
        array_0 = [[None for x in range(4)] for x in range(4)]
        for i in range(4):
            for j in range(4):
                array_0[i][j] = self.XOR(self.statearray[i][j],
                                         key_set[i][j])
        #round1-9
        cur_state = self.transpose(array_0) 
        for i in range(1,10):
            #byte_substitution
            for j in range(len(cur_state)):
                for k in range(len(cur_state[0])):
                    alph = cur_state[j][k]
                    cur_state[j][k] = self.sbox[int(alph[0],16)][int(alph[1],16)]
        
            #circular_left_shift_row            
            for m in range(len(cur_state)):
                cur_state[m] = cur_state[m][m:] + cur_state[m][:m]
                
            #mix_column
            after_mix = [['00']*len(cur_state[0]) for i in range(len(self.mix_matrix))]
            #construct after matrix(matrix multiplication)
            for row in range(len(self.mix_matrix)):
                for col in range(len(cur_state[0])):
                    for k in range(len(self.mix_matrix[0])): 
                        after_mix[row][col] = self.XOR(after_mix[row][col],
                                                       self.GF_multiply(self.mix_matrix[row][k],cur_state[k][col]))              
            #adding_roundkey
            rkey_matrix = self.transpose([key_set[4*i+m] for m in range(4)])
            state_mat = [[None for x in range(4)] for x in range(4)]
            for i in range(4):
                for j in range(4):
                    state_mat[i][j] = self.XOR(after_mix[i][j],rkey_matrix[i][j])
            
            cur_state = state_mat
        
        #round10(no mix_column)
         #byte_substitution
        for j in range(len(cur_state)):
            for k in range(len(cur_state[0])):
                alph = cur_state[j][k]
                cur_state[j][k] = self.sbox[int(alph[0],16)][int(alph[1],16)]
        
         #circular_left_shift_row            
        for m in range(len(cur_state)):
            cur_state[m] = cur_state[m][m:] + cur_state[m][:m]
        
         #adding_roundkey
        rkey_matrix = self.transpose([key_set[4*10+m] for m in range(4)])
        state_mat = [[None for x in range(4)] for x in range(4)]
        for i in range(4):
            for j in range(4):
                state_mat[i][j] = self.XOR(cur_state[i][j],rkey_matrix[i][j])
            
        cur_state = state_mat
        
        return ''.join([''.join(i) for i in self.transpose(cur_state)])#cur_state  
    
        
plaintext = '54776F204F6E65204E696E652054776F'   
key = '5468617473206D79204B756E67204675'
b = aes_encrypt(plaintext,key)
(b.generate())


Importing Jupyter notebook from AES_key_schedule.ipynb


'29C3505F571420F6402299B31A02D73A'